# Lesson 1: Router Engine

Welcome to Lesson 1.

To access the `requirements.txt` file, the data/pdf file required for this lesson and the `helper` and `utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

## Environment Setup
Recommend setting up an environment specifically for this tutorial.  It's always a good idea to create a new environment for each and every project.  I used Anaconda Navigator to create a new "llamindex" environment.  If using Anaconda Navigator make sure you install a version of Jupyter notebooks that's lower than 7.0...unless you're already there or not afraid.

## Setup

In [1]:
### uncomment to pip install
#!pip install python-dotenv==1.0.0

In [2]:
### uncomment to pip install
#!pip install llama-index==0.10.27

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 61.0/61.0 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB ? eta 0:00:00
     ---------------------------------------- 0.0/108.5 kB ? eta -:--:--
     -------------------------------------- 108.5/108.5 kB 6.6 MB/s eta 0:00:00
  Using cached certifi-2024.2.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB ? eta 0:00:00
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
   ---------------------------------------- 0.0/15.4 MB ? eta -:--:--
   ---------------------------------------- 0.2/15.4 MB 5.1 MB/s eta 0:00:03
   - -------------------------------------- 0.5/15.4 MB 6.7 MB/s 

In [3]:
### uncomment to pip install
#!pip install llama-index-llms-openai==0.1.15

In [4]:
### uncomment to pip install
#!pip install llama-index-embeddings-openai==0.1.7

  Attempting uninstall: llama-index-embeddings-openai
    Found existing installation: llama-index-embeddings-openai 0.1.10
    Uninstalling llama-index-embeddings-openai-0.1.10:
      Successfully uninstalled llama-index-embeddings-openai-0.1.10


In [5]:
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

In [6]:
import nest_asyncio

nest_asyncio.apply()

## Load Data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

In [7]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

## Define LLM and Embedding model

In [8]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [9]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

## Define Summary Index and Vector Index over the Same Data

In [10]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

## Define Query Engines and Set Metadata

In [11]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [12]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

## Define Router Query Engine

In [13]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [14]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: Useful for summarization questions related to MetaGPT.
The document introduces MetaGPT, a meta-programming framework that enhances multi-agent systems based on Large Language Models (LLMs) through role specialization, workflow management, and efficient communication mechanisms. It outperforms existing approaches in software development tasks, with potential future enhancements like self-improvement mechanisms and multi-agent economies. The document details the development process of a software application named "Drawing App" using MetaGPT, outlining the roles of various agents in the software development lifecycle. It discusses the use of Python libraries for GUI creation and testing, emphasizing the importance of team members being familiar with the project's libraries. Additionally, it explores MetaGPT's performance in generating executable code, addressing challenges like efficiently using context and reducing hallucinations. Ethical concerns related to Met

In [15]:
print(len(response.source_nodes))

34


In [16]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print(str(response))

Selecting query engine 0: Useful for summarization questions related to MetaGPT.
Agents share information with other agents through a shared message pool and a subscription mechanism. This shared message pool allows agents to publish structured messages and access messages from other agents transparently. The subscription mechanism enables agents to subscribe to relevant messages based on their profiles, ensuring they can extract necessary information efficiently. This structured communication interface enhances communication efficiency by allowing agents to focus on task-related information based on their roles.


## Let's put everything together
The code you ran through above is moved to get_router_query_engine.  Nice.  All you need to do now is send in the document name.

In [17]:
from utils import get_router_query_engine

query_engine = get_router_query_engine("metagpt.pdf")

In [18]:
response = query_engine.query("Tell me about the ablation study results?")
print(str(response))

Selecting query engine 1: Ablation study results are specific context from the MetaGPT paper, making choice 2 the most relevant..
The ablation study results provide insights into the impact of removing certain components or features from a system or model. This analysis helps in understanding the contribution and significance of individual elements towards the overall performance or functionality of the system.
